# 提升算法

![Adaboost](../images/提升算法/Adaboost.jpg)
![Adaboost](../images/提升算法/Adaboost2.jpg)
![Adaboost](../images/提升算法/Adaboost3.jpg)
![Adaboost](../images/提升算法/Adaboost4.jpg)
![Adaboost](../images/提升算法/Adaboost5.jpg)
![Adaboost](../images/提升算法/Adaboost6.jpg)
![Adaboost](../images/提升算法/Adaboost7.jpg)
![Adaboost](../images/提升算法/Adaboost8.jpg)
![Adaboost](../images/提升算法/Adaboost9.jpg)
![Adaboost](../images/提升算法/Adaboost10.jpg)
![Adaboost](../images/提升算法/Adaboost11.jpg)
![Adaboost](../images/提升算法/Adaboost12.jpg)

![Adaboost](../images/提升算法/提升树.jpg)
![Adaboost](../images/提升算法/提升树2.jpg)
![Adaboost](../images/提升算法/提升树3.jpg)
![Adaboost](../images/提升算法/提升树4.jpg)
![Adaboost](../images/提升算法/提升树5.jpg)
![Adaboost](../images/提升算法/提升树6.jpg)
![Adaboost](../images/提升算法/提升树7.jpg)

![Adaboost](../images/提升算法/梯度提升树.jpg)
![Adaboost](../images/提升算法/梯度提升树2.jpg)
![Adaboost](../images/提升算法/梯度提升树3.jpg)

In [1]:
import time
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
data = load_breast_cancer()
X, y = data['data'], data['target']
train_X, test_X, train_y, test_y = train_test_split(X, y)
print('train size: %s' % len(train_X))
print('test size: %s' % len(test_X))

train size: 426
test size: 143


In [3]:
class Adaboost:
    def __init__(self, train_X, train_y, num_trees=10):
        self.train_X = train_X
        self.train_y = train_y
        self.m, self.n = train_X.shape
        self.num_trees = num_trees
        self.weight_distribution = np.array([1 / self.m] * self.m)
        self.__train()
        
    def __calcErrorGx(self, feature_column, split, positive=True):
        error = 0
        x = self.train_X[:, feature_column]
        y = self.train_y
        predicts = []
        if positive:
            L = 0
            H = 1
        else:
            L = 1
            H = 0
        for i in range(self.m):
            if x[i] < split:
                predicts.append(L)
                if y[i] != L:
                    error += self.weight_distribution[i]
                    
            else:
                predicts.append(H)
                if y[i] != H:
                    error += self.weight_distribution[i]
                    
        return np.array(predicts), error
                    
    def __createSingleTree(self):
        tree = {}
        tree['error'] = 1
        for i in range(self.n):
            for split in np.unique(self.train_X[:, i]):
                for is_positive in [True, False]:
                    Gx, error = self.__calcErrorGx(i, split, is_positive)
                    if error < tree['error']:
                        tree['error'] = error
                        tree['split'] = split
                        tree['is_positive'] = is_positive
                        tree['Gx'] = Gx
                        tree['feature'] = i
        return tree
        
    def __train(self):
        trees = []
        for i in range(self.num_trees):
            tree = self.__createSingleTree()
            alpha = 1/2 * np.log((1-tree['error']) / tree['error'])
            Gx = tree['Gx']
            
            self.weight_distribution = np.multiply(
                self.weight_distribution, np.exp(-alpha * np.multiply(self.train_y, Gx))) / sum(self.weight_distribution)
            tree['alpha'] = alpha
            trees.append(tree)
        self.trees = trees
        
    def __predict(self, x):
        res = 0
        for tree in self.trees:
            split = tree['split']
            is_positive = tree['is_positive']
            feature = tree['feature']
            alpha = tree['alpha']
            if is_positive:
                if x[feature] > split:
                    pred = 1
                else:
                    pred = 0
                    
            else:
                if x[feature] > split:
                    pred = 0
                else:
                    pred = 1
            res += alpha * pred
            
        return 1 if res > 0.5 else 0
    
    def predict(self, test_X):
        preds = []
        for i in test_X:
            preds.append(self.__predict(i))
        return preds

In [4]:
adaboost = Adaboost(train_X, train_y)

In [5]:
print('算法准确率: %s' %(sum(adaboost.predict(test_X) == test_y) / len(test_y)))

算法准确率: 0.916083916083916
